Note: this notebook is set up to run with the env.yml containing the name 'polaris_datasets'

# Dataset creation with [Polaris](https://github.com/polaris-hub/polaris)
The first step of creating a benchmark is to set up a standard dataset which allows accessing the curated dataset (which has been demonstrated in <01_AZ_PPB_data_curation.ipynb>), and all necessary information about the dataset such as data source, description of endpoints, units etc. 

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import pathlib

import pandas as pd
import datamol as dm

# polaris dataset
from polaris.dataset import Dataset, ColumnAnnotation
from polaris.utils.types import HubOwner


root = pathlib.Path("__file__").absolute().parents[3]
os.chdir(root)
sys.path.insert(0, str(root))
from utils.docs_utils import load_readme

/Users/lu.zhu/miniconda3/envs/po_dataset/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Get the owner and organization
org = "polaris"
data_name = "PPB_clearance"
dirname = dm.fs.join(root, f"org-{org}", "astra_zeneca", data_name)
gcp_root = f"gs://polaris-public/polaris-recipes/org-{org}/astra_zeneca/{data_name}"

owner = HubOwner(slug=org, type="organization")
owner

HubOwner(slug='polaris', external_id=None, type='organization')

In [3]:
BENCHMARK_DIR = f"{gcp_root}/benchmarks"
DATASET_DIR = f"{gcp_root}/datasets"
FIGURE_DIR = f"{gcp_root}/figures"

## Load existing data

In [4]:
PATH = f"{gcp_root}/data/curation/{data_name}_curated.csv"
table = pd.read_csv(PATH)
table.columns

Index(['SMILES', 'PPB', 'log_unbound_PPB', 'MOL_smiles', 'MOL_molhash_id',
       'MOL_molhash_id_no_stereo', 'MOL_num_stereoisomers',
       'MOL_num_undefined_stereoisomers', 'MOL_num_defined_stereo_center',
       'MOL_num_undefined_stereo_center', 'MOL_num_stereo_center',
       'MOL_undefined_E_D', 'MOL_undefined_E/Z', 'OUTLIER_log_unbound_PPB',
       'AC_log_unbound_PPB'],
      dtype='object')

### Below we specify the meta information of data columns

In [5]:
# Additional meta-data on the column level
annotations = {
    "MOL_smiles": ColumnAnnotation(
        description="Molecule SMILES string", modality="molecule"
    ),
    "log_unbound_PPB": ColumnAnnotation(
        description="Log of percent unbound to whole human plasma by equilibrium dialysis.",
        user_attributes={"unit": "percent", "organism": "human"},
    ),
}

### Define `Dataset` object

In [6]:
version = "v1"
dataset_name = f"{data_name}-{version}"

In [7]:
dataset = Dataset(
    # The table is the core data-structure required to construct a dataset
    table=table[annotations.keys()],
    # Additional meta-data on the dataset level.
    name=dataset_name,
    description="Log percent of compound unbound to whole human plasma (PPB) experiment data released by AstraZeneca",
    source="https://www.ebi.ac.uk/chembl/document_report_card/CHEMBL3301365/",
    annotations=annotations,
    tags=["ADME"],
    owner=owner,
    license="CC-BY-SA-4.0",
    user_attributes={"year": "2016"},
    readme=load_readme("org-Polaris/astra_zeneca/PPB_clearance/AZ_PPB_readme.md"),
    curation_reference="https://github.com/polaris-hub/polaris-recipes/org-Polaris/astra_zeneca/PPB_clearance/01_AZ_PPB_data_curation.ipynb",
)

In [8]:
# save the dataset to GCP
SAVE_DIR = f"{DATASET_DIR}/{dataset_name}"
dataset_path = dataset.to_json(SAVE_DIR)
dataset_path

'gs://polaris-public/polaris-recipes/org-polaris/astra_zeneca/PPB_clearance/datasets/PPB_clearance-v1/dataset.json'

In [9]:
# from polaris.hub.client import PolarisHubClient
# client = PolarisHubClient()
# client.login()

# client.upload_dataset(dataset=dataset, access="private", owner=owner)

2024-07-10 01:57:27.974 | INFO     | polaris.hub.client:login:285 - You are already logged in to the Polaris Hub as  (lu@valencediscovery.com). Set `overwrite=True` to force re-authentication.
2024-07-10 01:57:30.113 | SUCCESS  | polaris.hub.client:upload_dataset:631 - Your dataset has been successfully uploaded to the Hub. View it here: https://polarishub.io/datasets/polaris/PPB_clearance-v1


{'id': 'QGaHW27Q1pnog2SgVQE0w',
 'createdAt': '2024-07-10T05:57:28.696Z',
 'deletedAt': None,
 'name': 'ppb-clearance-v1',
 'slug': 'ppb-clearance-v1',
 'description': 'Log percent of compound unbound to whole human plasma (PPB) experiment data released by AstraZeneca',
 'tags': ['ADME'],
 'userAttributes': {'year': '2016'},
 'access': 'private',
 'isCertified': False,
 'polarisVersion': 'dev',
 'readme': "## Background\nThis is part of a release of experimental data determined at AstraZeneca on a set of compounds in the following assays: pKa, lipophilicity (LogD7.4), aqueous solubility, plasma protein binding (human, rat, dog , mouse and guinea pig), intrinsic clearance (human liver microsomes, human and rat hepatocytes). \n\n## Assay Information:\nDuring a trip through the circulatory system, a drug can bind to many different proteins within the blood plasma. This can affect a drug's efficacy - only the unbound fraction of the drug will exert pharmacological effects. Equilibrium dial